In [ ]:
## RAG 없이 
# %pip install -qU python-docx
# %pip install -qU tiktoken
# %pip install -qU chromadb
from openai import OpenAI
from docx import Document
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import os
import chromadb
import tiktoken

## 문서 내용 읽기

document = Document('law_1.docx')
for index, paragraph in enumerate(document.paragraphs):
    print(f"{index}. {paragraph.text}")

    if index ==  4 :
        break

## 읽어온 단락을 하나의 문서로 만들기
full_text = ""

for paragraph in document.paragraphs:
    full_text += f'{paragraph.text}'

## 문서 쪼개기  
encoding_model = tiktoken.encoding_for_model('gpt-4o')

## 토큰화
encoding = encoding_model.encode(full_text)
decoding = encoding_model.decode([9516, 11734, 5537])
def split_text(full_text, chunk_size):
    encoding_model=tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoding_model.encode(full_text)
    total_token_count = len(total_encoding)

    test_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoding = encoding_model.decode(chunk)
        test_list.append(decoding)

    return test_list
    
chunk_list = split_text(full_text, 1500)

## 벡터 데이터베이스에 저장  
## 임베딩 모델 지정

OPEN_API_KEY = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=OPEN_API_KEY, model_name='text-embedding-3-large')
chroma_client = chromadb.Client()
collection_name = 'law_collection'
law_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)
id_list = []

for index in range(len(chunk_list)):
    id_list.append(f"{index}")

law_collection.add(documents=chunk_list, ids=id_list)
collections = chroma_client.list_collections()

## 유사도 검색  
query = "전세사기 피해 구제에 대해 알려주세요"
retrieved_docs = law_collection.query(query_texts=query, n_results=3)

client = OpenAI()

response = client.responses.create(
    model="gpt-4o",
    instructions=f"당신은 전세사기 피해 법률 전문가입니다. {retrieved_docs['documents'][0]}를 참고해 사용자의 질문에 답변해주세요.",
    input=query,
)
print(response.output_text)

In [ ]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "developer",
            "content": f"당신은 전세사기 피해 법률 전문가입니다. {retrieved_docs['documents'][0]}를 참고해 사용자의 질문에 답변해주세요."
        },
        {
            "role": "user",
            "content": "전세사기 피해 구제에 대해 알려주세요"
        }
    ]
)
print(response.output_text)